In [ ]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
load_dotenv()
import os
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Initialize the Pinecone client
api_key="8ad9cf35-9867-4ce2-99b1-42a7e29e44a0"
pc = Pinecone(api_key=api_key)
index_name = "discovery-test"

# Connect to the index
index = pc.Index(index_name)

In [ ]:
stats = index.describe_index_stats()
stats

In [ ]:
def _get_all_vector_ids(
    pinecone_index: str,
    namespace="",
    filter={},
):
    try:
        index = pc.Index(pinecone_index)
        stats = index.describe_index_stats()
        vector_count = stats.namespaces.get(namespace, {}).get("vector_count", 0)
        # Namespace doesn't exist
        if vector_count == 0:
            return []
        
        vector_counts = []
        max_value = 10
        
        while vector_count > 0:
            if vector_count >= max_value:
                vector_counts.append(max_value)
                vector_count -= max_value
            else:
                vector_counts.append(vector_count)
                vector_count = 0
        result = []
        for count in vector_counts:
            query = index.query(
                top_k=count,
                vector=[0] * 1536,  # embedding dimension
                namespace=namespace,
                include_values=False,
                include_metadata=False,
                filter={**filter, "vec_id":{"$nin":result}},
            )
            current_vecs = [vector["id"] for vector in query["matches"]]
            result.extend(current_vecs)

            for id in current_vecs:
                #Updating Metadata Syntax
                index.update(
                    id=id, 
                    set_metadata={"vec_id": id}, 
                    namespace=""
                )
        return query
    except:
        raise Exception("Unable to get all vector ids")
    
res = _get_all_vector_ids("discovery-test")
res